In [168]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [169]:
tf.disable_eager_execution()
tf.executing_eagerly()

False

In [170]:
df=pd.read_csv(r'C:\Users\PC\Dropbox\renana rimon\semester 5\deep learning\StrokeProject\brain_stroke.csv')
df.head()


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [171]:
df['gender'] = df['gender'].map({'Female':1,'Male':0})
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df['work_type'] = df['work_type'].map({'Private': 0, 'Self-employed': 1, 'Govt_job':2, 'children':3})
df['Residence_type'] = df['Residence_type'].map({'Urban': 1, 'Rural':0})
df['smoking_status'] = df['smoking_status'].map({'formerly smoked':0, 'never smoked':1, 'smokes':2, 'Unknown':3})
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,1,0,1,228.69,36.6,0,1
1,0,80.0,0,1,1,0,0,105.92,32.5,1,1
2,1,49.0,0,0,1,0,1,171.23,34.4,2,1
3,1,79.0,1,0,1,1,0,174.12,24.0,1,1
4,0,81.0,0,0,1,0,1,186.21,29.0,0,1


In [172]:
tf.random.set_random_seed(22)

df = df.sample(frac = 1)

In [173]:
print("data shape:")
train_dataset = df.sample(frac=0.75, random_state=1)
test_dataset = df.drop(train_dataset.index)
print("train: ",train_dataset.shape, "\ntest: ", test_dataset.shape)
x_train, y_train = train_dataset.iloc[:,:-1], train_dataset.iloc[:,-1:]
x_test, y_test = test_dataset.iloc[:,:-1], test_dataset.iloc[:,-1:]
print("x_train: ",x_train.shape, "\ny_train: ", y_train.shape)
print("x_test: ",x_test.shape, "\ny_test: ", y_test.shape)

data shape:
train:  (3736, 11) 
test:  (1245, 11)
x_train:  (3736, 10) 
y_train:  (3736, 1)
x_test:  (1245, 10) 
y_test:  (1245, 1)


In [181]:
# def Norm(x_data):
#     mean = np.mean(x_data, axis=0)
#     std = np.std(x_data, axis=0)
#     return (x_data - mean)/std

In [182]:
# x_train = Norm(x_train)
# x_test = Norm(x_test)

In [183]:
# x_train.head()

In [184]:
features = x_train.shape[1]
eps = 1e-12
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([features,1]))
b = tf.Variable(tf.zeros([1]))

In [185]:
pred = tf.nn.sigmoid(tf.matmul(x,W) + b)
loss1 = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=pred)
loss= tf.reduce_mean(loss1)

In [186]:
update = tf.train.GradientDescentOptimizer(0.000001).minimize(loss)

In [187]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

In [188]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(0,10000):
	sess.run(update, feed_dict = {x:x_train, y_:y_train})

In [189]:
# pred_test = np.empty_like(y_test)
# for i in range(x_test.shape[0]):
#     if logistic_fun(np.matmul(x_test.iloc[i],sess.run(W)) + sess.run(b)) > 0.5:
#         pred_test[i] = 1
#     else:
#         pred_test[i] = 0

In [190]:
from sklearn.metrics import accuracy_score

In [191]:
# accuracy_score(y_test, pred_test)

In [192]:
# pred_train = np.empty_like(y_train)
# for i in range(x_train.shape[0]):
#     p = logistic_fun(np.matmul(x_train.iloc[i],sess.run(W)) + sess.run(b))
#     # print(p)
#     if p > 0.5:
#         pred_train[i] = 1
#     else:
#         pred_train[i] = 0

In [193]:
# accuracy_score(y_train, pred_train)

In [194]:
def predict(x_data, y_data):
    y_pred = []
    answer = []
    for j in range(y_data.shape[0]):
        p=logistic_fun(np.matmul(x_data.iloc[j],sess.run(W)) + sess.run(b))
        answer.append(p)
        if p > 0.07:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return np.array(y_pred), np.array(answer)

In [195]:
pred_test, ans = predict(x_test, y_test)
print("train accuracy:",accuracy_score(y_test, pred_test),
      "\nf1_score: ", f1_score(y_test, pred_test),
      "\nrecall: ", recall_score(y_test, pred_test),
      "\nprecision: ", precision_score(y_test, pred_test))

train accuracy: 0.8987951807228916 
f1_score:  0.015625 
recall:  0.014285714285714285 
precision:  0.017241379310344827


In [196]:
ans

array([[0.01708656],
       [0.00671319],
       [0.00141614],
       ...,
       [0.03455209],
       [0.03251601],
       [0.02538818]])

In [197]:
ans.min()

7.673296116472097e-05

In [198]:
ans.max()

0.11718953196947617

In [199]:
pred_test.sum()

58

In [200]:
y_test.sum()

stroke    70
dtype: int64

In [201]:
y_train

,stroke
4004,0
374,0
2165,0
3240,0
1006,0
...,...
3923,0
3094,0
3432,0
2670,0


In [202]:
y_train.sum()

stroke    178
dtype: int64

In [204]:
y_test.sum()

stroke    70
dtype: int64